In [1]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner,set_tracing_disabled,OpenAIChatCompletionsModel
import os
from os.path import join

In [ ]:
input_folder_path = '..\inputs'
yapping_info_file_path = 'yapping_info.txt'

yapping_file = join(input_folder_path, yapping_info_file_path)

print(yapping_file)

In [3]:
load_dotenv(override=True)

True

In [4]:
perplexcity_model = "sonar-pro"
ollama_model = 'qwen3:8b'

In [5]:
perplexcity_api_key =  os.getenv("PERPLEXCITY_API_KEY")
print(perplexcity_api_key[:4])

pplx


In [6]:
ollama = AsyncOpenAI(
    api_key = "ollama",
    base_url = "http://127.0.0.1:11434/v1"
)

In [7]:

perplexcity = AsyncOpenAI(
    api_key=perplexcity_api_key,
    base_url = "https://api.perplexity.ai"
)

In [8]:
set_tracing_disabled(True)

In [9]:
ollama_agent = OpenAIChatCompletionsModel(
        model=ollama_model,
        openai_client=ollama)

In [10]:
perplexcity_agent = OpenAIChatCompletionsModel(
        model=perplexcity_model,
        openai_client=perplexcity)

In [11]:
def process_ai_response(response):
    if "<think>" in response and "</think>" in response:
        # Extract everything cleanly
        before_think = response.split("<think>")[0]
        think_content = response.split("<think>")[1].split("</think>")[0]
        after_think = response.split("</think>")[1]

        # Format the think block for display
        formatted_think = f"> 💭 *{think_content.strip()}*"

        # Combine all parts
        return after_think.strip()
    else:
        return response

In [12]:
with open(yapping_file) as f:
    yapping_file = f.read()

print(yapping_file)

I am working as a crypto X(formarly twitter) content creator. 
when a new crypto project release, they will have a marketing campaign in a platform called kaito yapps

Kaito Yaps is an AI-driven crypto rewards program that incentivizes sharing smart, original, and engaging crypto insights on X.

Rewards users for high-quality, insightful crypto posts via "Yap Points," with engagement, relevance, and originality prioritized.

Uses AI to distinguish genuine contributions from spam, enabling fair recognition and potential future airdrops.

Leaderboards track influential contributors and foster healthy competition in the crypto information ecosystem.

Aims to build the world's first tokenized attention economy for web3 knowledge sharing.

currently I am yapping on OpenLedger project with twitter or X handle @OpenLedgerHQ and also for another project called Billions with X or twitter handle as @billions_ntwk

so, in every good morning and good night wish I need to add content about the same

In [13]:
good_morning_wish_instruct = 'you are working as a good morning wishing agent who wish good morning to crypto twitter. you will do the following : \n \
    - will wish a energetic good morning message to the crypto fam. \n \
    - the wish should sound like humanly. \n \
    - should never use the words like low key. \n \
    - can add some genz wibe to the wish. or crypto meme vibe. \n \
    - add emojis as needed, but not too many emojis \n \
    - never use \'-\'.'

ai_info_instruct = 'you are working as a agent who shared latest news about the AI advances. you will do the following : \n \
    - search internet to find latest news about AI world. get the information about agentic AI or AI models and LLM.\n \
    - find the best info that is relevent and latest from the gathered information. \n \
    - you will then summarize the information in minimal words, 20 words. \n \
    - you will only get one news or info to the user which you think is the most relevant to new information about AI. \n \
    - you will present the news in a energetic manner, catchy opening, tweet meme culture. \n \
    - you should never use the words like leveled up, -, low key'

yapping_info_instruct = f'you are working as a helper who identify the project the user is yapping about. the yapping info is presented to you, you will then make a catchy statement to ask the followers of the user to yapp about the project the user is yapping about. \n \
    the yapping information is as follows : \n \n \
        {yapping_file} \n \
            - you will find the relevant twitter handle from the informatin given to you. \n \
            - you will then make a catchy statement which ask the followers of the user to yapp about the project which user is yapping about. \n \
            - the statement should contain all the @ or tweeter handle of the project mentoined in the project. \n \
            - for example, a message can be like : Fam, keep yapping about @<twitter handle of project 1> and @<twitter handle of project 2>. this is just an example. the original one should have the twitter handle detail along with energetic message. \n \
            - don\'t add any info about time like good night, good morning. just the yapping message is enough. \n \
            - remove anything in <think> and </think> tags. remove any text inside the tag and tag itself and present the info to the user'

print(good_morning_wish_instruct + "\n")
print(ai_info_instruct)
print(yapping_info_instruct)

you are working as a good morning wishing agent who wish good morning to crypto twitter. you will do the following : 
     - will wish a energetic good morning message to the crypto fam. 
     - the wish should sound like humanly. 
     - should never use the words like low key. 
     - can add some genz wibe to the wish. or crypto meme vibe. 
     - add emojis as needed, but not too many emojis 
     - never use '-'.

you are working as a agent who shared latest news about the AI advances. you will do the following : 
     - search internet to find latest news about AI world. get the information about agentic AI or AI models and LLM.
     - find the best info that is relevent and latest from the gathered information. 
     - you will then summarize the information in minimal words, 20 words. 
     - you will only get one news or info to the user which you think is the most relevant to new information about AI. 
     - you will present the news in a energetic manner, catchy opening, tw

In [14]:
good_morning_wish_agent = Agent('good_morning_wish_agent' , instructions=good_morning_wish_instruct, model=perplexcity_agent)

In [15]:
ai_info_agent = Agent('ai_info',ai_info_instruct, instructions=ai_info_instruct, model=perplexcity_agent)

In [16]:
yapping_info_agent = Agent("yapp_info", instructions=yapping_info_instruct, model=ollama_agent)

In [17]:
good_morning_wish_tool = good_morning_wish_agent.as_tool(tool_name='good_morning_wish', tool_description='used to generate a good morning wish to twitter crypto users')
ai_info_tool = ai_info_agent.as_tool(tool_name='ai_info_finder', tool_description='give latest information in AI world')
yapp_info_tool = yapping_info_agent.as_tool(tool_name='yapp_info' , tool_description='use this tool to get yapp info and the project the user is yapping about.')

print(f'{good_morning_wish_tool} \n {ai_info_tool} \n {yapp_info_tool}')

FunctionTool(name='good_morning_wish', description='used to generate a good morning wish to twitter crypto users', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'good_morning_wish_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001237F8AB240>, strict_json_schema=True, is_enabled=True) 
 FunctionTool(name='ai_info_finder', description='give latest information in AI world', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_info_finder_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001237F8AB600>, strict_json_schema=True, is_enabled=True) 
 FunctionTool(name='yapp_info', description='use this tool to get yapp info and the project th

In [18]:
tools = [good_morning_wish_tool,ai_info_tool,yapp_info_tool]
print(tools)

[FunctionTool(name='good_morning_wish', description='used to generate a good morning wish to twitter crypto users', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'good_morning_wish_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001237F8AB240>, strict_json_schema=True, is_enabled=True), FunctionTool(name='ai_info_finder', description='give latest information in AI world', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'ai_info_finder_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001237F8AB600>, strict_json_schema=True, is_enabled=True), FunctionTool(name='yapp_info', description='use this tool to get yapp info and the project the

In [19]:
tweet_creator_master_instruct = ' you are working as a agent who helps the user to make tweet. user is focusing on crypto twitter. you will do the following : \n \
    - add the good morning wish given to you. \n \
    - then you will add the ai information given to you. \n \
    - next you will add the yapping information presented to you. \n \
    - conclude the tweet in a funny way. the tweet is a good morning tweet. so conclude the tweet in a funny way which fit in the context and relatable to people.'

print(tweet_creator_master_instruct)

 you are working as a agent who helps the user to make tweet. user is focusing on crypto twitter. you will do the following : 
     - add the good morning wish given to you. 
     - then you will add the ai information given to you. 
     - next you will add the yapping information presented to you. 
     - conclude the tweet in a funny way. the tweet is a good morning tweet. so conclude the tweet in a funny way which fit in the context and relatable to people.


In [20]:
tweet_creator_master_agent = Agent('tweet_draft_master' , instructions=tweet_creator_master_instruct, model = perplexcity_agent, handoff_description='convert the given information to a meaningful crypto tweet.')

In [21]:
tweet_cordinator_instruct = 'you are an  agent who cordinate  good morning tweet creaion for the user. user is focusing on crypto tweetitter. AS a tweet drafting agent, you will do the following : \n \
    - get the good morning wish from good_morning_wish tool. \n \
    - process the information you got. \n \
    - get latest ai info from ai_info_finder. \n \
    - process the information you got. \n \
    - use the tool yapp_info to get the message that you need to add for the yapping project user is doing. this tool will have content in <think> and </think> tags. don\'t consider the tags and the content inside the tags. use the other info shared by the tool for the yapp informaiton. \n \
    - once you get all the information, you will hand over the information to tweet tweet_draft_master tool'

print(tweet_cordinator_instruct)

you are an  agent who cordinate  good morning tweet creaion for the user. user is focusing on crypto tweetitter. AS a tweet drafting agent, you will do the following : 
     - get the good morning wish from good_morning_wish tool. 
     - process the information you got. 
     - get latest ai info from ai_info_finder. 
     - process the information you got. 
     - use the tool yapp_info to get the message that you need to add for the yapping project user is doing. this tool will have content in <think> and </think> tags. don't consider the tags and the content inside the tags. use the other info shared by the tool for the yapp informaiton. 
     - once you get all the information, you will hand over the information to tweet tweet_draft_master tool


In [22]:
tweet_cordinator_agent = Agent('tweet_cordinator', instructions = tweet_cordinator_instruct, model = ollama_agent, tools=tools, handoffs=[tweet_creator_master_agent])

In [45]:
result = await Runner.run(tweet_cordinator_agent, 'draft me the good morning tweet')


In [46]:
print(result.final_output)

Good morning, Crypto fam! ☀️ Here’s your daily nugget: AI is taking things up a notch—Bittensor (TAO) is gearing up for its first halving and a dTAO upgrade that’s making subnet investments go wild. Meanwhile, everyone’s still yapping about @OpenLedgerHQ & @billions_ntwk and their latest moves! Stay sharp out there... my portfolio’s got more drama than a Reality TV reunion. 💸📈 #Crypto #AI #Web3
